In [1]:
import os
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import matplotlib.pyplot as plt

In [2]:
image_path = (r"C:\Users\rajan\Downloads\NewGen\Pan Card\train")

In [3]:
data = DataLoader.from_folder(image_path)
train_data, rest_data = data.split(0.8)

INFO:tensorflow:Load image with size: 757, num_label: 2, labels: Not Pan Card, Pan Card.


In [4]:
validation_data, test_data = rest_data.split(0.5)

In [5]:
model = image_classifier.create(train_data, model_spec=model_spec.get('efficientnet_lite0'), validation_data=validation_data, epochs = 20)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None


C:\Users\rajan\Anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/20
18/18 [==============================] - 20s 987ms/step - loss: 0.3852 - accuracy: 0.8611 - val_loss: 0.2501 - val_accuracy: 1.0000
Epoch 2/20
18/18 [==============================] - 17s 939ms/step - loss: 0.2641 - accuracy: 0.9965 - val_loss: 0.2359 - val_accuracy: 1.0000
Epoch 3/20
18/18 [==============================] - 17s 924ms/step - loss: 0.2291 - accuracy: 0.9948 - val_loss: 0.2062 - val_accuracy: 1.0000
Epoch 4/20
18/18 [==============================] - 17s 923ms/step - loss: 0.2134 - accuracy: 1.0000 - val_loss: 0.2055 - val_accuracy: 1.0000
Epoch 5/20
18/18 [==============================] - 17s 937ms/step - loss: 0.2105 - accuracy: 1.0000 - val_loss: 0.2039 - val_accuracy: 1.0000
Epoch 6/20
18/18 [==============================] - 17s 923ms/step - loss: 0.2096 - accuracy: 1.0000 - val_loss: 0.2036 - val_accuracy: 1.0000
Epoch 7/20
18/18 [==============================] - 17s 925ms/step - loss: 0.2083 - accuracy: 0.9983 - val_loss: 0.2032 - val_accuracy: 1.0000

In [6]:
loss, accuracy = model.evaluate(test_data)

3/3 [==============================] - 2s 508ms/step - loss: 0.2025 - accuracy: 1.0000


In [7]:
model.export(export_dir='.')

INFO:tensorflow:Assets written to: C:\Users\rajan\AppData\Local\Temp\tmpq2txdjca\assets


INFO:tensorflow:Assets written to: C:\Users\rajan\AppData\Local\Temp\tmpq2txdjca\assets
C:\Users\rajan\Anaconda3\lib\site-packages\tensorflow\lite\python\convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in C:\Users\rajan\AppData\Local\Temp\tmpli_sq32l\labels.txt


INFO:tensorflow:Saving labels in C:\Users\rajan\AppData\Local\Temp\tmpli_sq32l\labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: .\model.tflite


TFLite

In [8]:
import tensorflow.lite as tflite
# Load TFLite model and allocate tensors.
interpreter = tflite.Interpreter(model_path=r"C:\Users\rajan\Downloads\model.tflite")
#allocate the tensors
interpreter.allocate_tensors()

In [9]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [130]:
import cv2
import numpy as np
image_path=(r"C:\Users\rajan\Downloads\NewGen\Pan Card\train\Not Pan Card\G701844_09-25_264.jpeg")
img = cv2.imread(image_path)
img = cv2.resize(img,(224,224))
#Preprocess the image to required size and cast
input_shape = input_details[0]['shape']
input_tensor= np.array(np.expand_dims(img,0))

In [131]:
input_index = interpreter.get_input_details()[0]["index"]
interpreter.set_tensor(input_index, input_tensor)
interpreter.invoke()
output_details = interpreter.get_output_details()

In [132]:
output_data = interpreter.get_tensor(output_details[0]['index'])
pred = np.squeeze(output_data)

In [133]:
print(pred)

[ 34 222]


In [134]:
class_ind = {
1 :'Pan Card',
0:'Not Pan Card'}

In [135]:
highest_pred_loc = np.argmax(pred)

In [136]:
document_name = class_ind[highest_pred_loc]
print(f"This Document is {document_name}")

This Document is Pan Card


In [137]:
print(np.argmax(pred))

1
